In [ ]:
import matplotlib
import matplotlib.pyplot as plt
from scipy import special
import numpy as np
from ipywidgets import *

Daten und Parameter

In [ ]:
#FUNCTIONS FOR COMPUTATION; ADS = ADVECTION, DISPERSION AND SORPTION - EVENTUALLY SET RETARDATION TO 1 FOR NO SORPTION
def BETA(v,D,lambd,R):
    beta=np.sqrt((v/(2*D))**2+(lambd*R)/D)
    return beta

def TERM0_ADSD(v,x,D):
    term0 = np.exp(v*x/(2*D))
    return term0

def TERM1_ADS (x,v,R,t,D,sign):
    term1 = special.erfc((x+sign*(v/R)*(t))/(2*np.sqrt((D*t/R))))
    return term1

def TERM1_ADSD(x,v,R,t,D,lambd,sign):
    term1=  special.erfc((x+sign*t*np.sqrt((v/R)**2+4*lambd*D/R))/(2*np.sqrt(D*t/R)))
    return term1

def TERM2_ADS(v,D,x):
    term2 = np.exp(v/D*x)
    return term2

def TERM2_ADSD(v,D,lambd,R,x,sign):
    beta = BETA(v,D,lambd,R)
    term2 = np.exp(sign*beta*x)
    return term2

def transport(l,f,t1,ci,c0,Q,n1,n2,D1,D2,R1,R2,lambd1,lambd2):
    
    # Data for plotting
    t0 = 1       #Startzeit
    dt = 1       #Zeitdiskretisierung
    r  = 0.1     #Radius der Säule
    #Berechnung Zwischenergebnisse
    A = np.pi*r**2
    q = Q/A
    v1 = q/n1
    v2 = q/n2

    #Festlegung Zeitbereich
    t = np.arange(t0, t1, dt)

    #Berechnung Konzentration - Klammerterme
    #Set fraction of distance
    i = 0
    result1 = []
    result2 = []
    time = []

    #compute distance based on length and fraction
    x = f*l

    #compute concentration  
    for t in range(t0, t1, dt):      
        # ADVECTION-DISPERSION WITH OPTIONAL SORPTION / WITHOUT DECAY
        if lambd1 == 0:
            Term1m = TERM1_ADS(x,v1,R1,t,D1,-1)
            Term1p = TERM1_ADS(x,v1,R1,t,D1,1)
            Term2 = TERM2_ADS(v1,D1,x)
            c1 = ci+((c0-ci)/2)*(Term1m+Term2*Term1p)
        # WITH ADDITIONAL DECAY
        else:
            Term0  = TERM0_ADSD(v1,x,D1)
            Term1m = TERM1_ADSD(x,v1,R1,t,D1,lambd1,-1)
            Term1p = TERM1_ADSD(x,v1,R1,t,D1,lambd1,1)
            Term2m = TERM2_ADSD(v1,D1,lambd1,R1,x,-1)
            Term2p = TERM2_ADSD(v1,D1,lambd1,R1,x,1)
            c1 = ci+(c0-ci)/2*Term0*(Term2m*Term1m+Term2p*Term1p)
            
        result1.append(c1)
        
# REPEAT FOR VARIANT 2
        # ADVECTION-DISPERSION WITH OPTIONAL SORPTION / WITHOUT DECAY
        if lambd2 == 0:
            Term1m = TERM1_ADS(x,v2,R2,t,D2,-1)
            Term1p = TERM1_ADS(x,v2,R2,t,D2,1)            
            Term2 = TERM2_ADS(v2,D2,x)
            c2 = ci+((c0-ci)/2)*(Term1m+Term2*Term1p)
        else:
            # WITH ADDITIONAL DECAY
            Term0  = TERM0_ADSD(v2,x,D2)
            Term1m = TERM1_ADSD(x,v2,R2,t,D2,lambd2,-1)
            Term1p = TERM1_ADSD(x,v2,R2,t,D2,lambd2,1)            
            Term2p = TERM2_ADSD(v2,D2,lambd2,R2,x,1)
            Term2m = TERM2_ADSD(v2,D2,lambd2,R2,x,-1)
            c2 = ci+(c0-ci)/2*Term0*(Term2m*Term1m+Term2p*Term1p)
            
        result2.append(c2)
        
        time.append(t)
        
# PLOT HERE        
    fig, ax = plt.subplots()
    ax.plot(time,result1)
    ax.plot(time,result2)
    plt.ylim(0, ci+c0)
    plt.xlim(0,t1)
    ax.set(xlabel='time', ylabel='conc.',
       title='A-D transport according to Ogata-Banks')
    ax.grid()
    plt.legend(('1', '2'), loc=4)
    
# COMMENTED THIS OUT - OPTIONAL SAVE IF INTENDED    
   # fig.savefig("test.png")
    plt.show()
    print('observation at x = ', '{:1.2f}'.format(x))
    
#THE FOLLOWING ALLOWS TO DEFINE THE RANGE OF SLIDERS DEPENDING ON OTHER SLIDER SETTINGS (HERE DISP AS FUNCTION OF LENGTH)
    
D1_widget = FloatSlider(value=1,min=0.005, max=10,step=0.1,description='DISP 1', readout_format='.3f' )
D2_widget = FloatSlider(value=1,min=0.005, max=10,step=0.1,description='DISP 2', readout_format='.3f' )
l_widget  = FloatSlider(value=50,min=1, max=100,step=1,description='LENGTH', readout_format='.2f' )

def update_D_range(*args):
    D1_widget.min = 0.00015 * l_widget.value
    D2_widget.min = 0.00015 * l_widget.value
    D1_widget.max = l_widget.value
    D2_widget.max = l_widget.value
    D1_widget.step = 0.01*l_widget.value
    D2_widget.step = 0.01*l_widget.value
    
l_widget.observe(update_D_range, 'value')
   
interact(transport,
         #DEFINE THE RANGE SUCH THAT NOT TOO MANY COMPUTATIONS ARE NECESSARY WHEN MOVING THE SLIDER!
         l=l_widget,
         f=widgets.FloatSlider(value=0.5,min=0.01, max=1,step=0.01,description='OBS (FRACTION OF LENGTH)', readout_format='.2f' ),
         t1 = widgets.IntSlider(value=3600, min = 60, max = 86400, step = 60, description = 'MAX TIME'),
         ci=widgets.FloatSlider(value=0,min=0,max=10,step=0.1, description='C initial',readout_format='.2f'),
         c0=widgets.FloatSlider(value=2,min=0, max=5,step=0.1,description='C0', readout_format='.2f' ),
         Q=widgets.FloatSlider(value=0.0005, min=0.00001, max=0.001, step=0.0001, description='Discharge Q',readout_format='.5f'),
         n1=widgets.FloatSlider(value=0.2,min=0.02, max=0.6, step=0.01, description='PORO 1',readout_format='.2f'),
         n2=widgets.FloatSlider(value=0.2,min=0.02, max=0.6, step=0.01, description='PORO 2', readout_format='.2f' ),         
         D1=D1_widget,
         D2=D2_widget,
         R1=widgets.FloatSlider(value=1, min=0.8, max=10,step=0.1,description='Retardation 1',readout_format='.2f'),
         R2=widgets.FloatSlider(value=1, min=0.8, max=10,step=0.1,description='Retardation 2',readout_format='.2f'),
         lambd1=widgets.FloatSlider(value=0,min=0,max=0.005,step=0.0005,description='LAMBDA 1',readout_format='.4f'),
         lambd2=widgets.FloatSlider(value=0,min=0,max=0.005,step=0.0005,description='LAMBDA 2',readout_format='.4f'),
        )